<a href="https://colab.research.google.com/github/mjsipes/stock_algorithms/blob/main/StockAlgorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
def perfectalgo(data, initial_investment, fee_percent, DEBUG):
    current_cash = initial_investment
    stocks_owned = 0
    fee_decimal = fee_percent / 100.0

    price = [0] * len(data)
    difference = [0] * len(data)
    pdifference = [0] * len(data)
    direction = ["-"] * len(data)
    action = ["HOLD"] * len(data)
    networth = [0] * len(data)

    for i in range(len(data)):
        price[i] = data["Close"].iloc[i]
        if i > 0:
            difference[i] = data["Close"].iloc[i] - data["Close"].iloc[i-1]
            pdifference[i] = (data["Close"].iloc[i] - data["Close"].iloc[i-1]) / data["Close"].iloc[i]
            direction[i] = "UP" if (data["Close"].iloc[i] - data["Close"].iloc[i-1]) > 0 else "DOWN"

            if i < len(data)-1 and data["Close"].iloc[i+1] > data["Close"].iloc[i]:
                if current_cash > 0:
                    action[i] = "BUY"
                    stocks_to_buy = current_cash / (data["Close"].iloc[i] * (1 + fee_decimal))
                    stocks_owned += stocks_to_buy
                    current_cash = 0

            elif i < len(data)-1 and data["Close"].iloc[i+1] < data["Close"].iloc[i]:
                if stocks_owned > 0:
                    action[i] = "SELL"
                    cash_from_sale = stocks_owned * data["Close"].iloc[i] * (1 - fee_decimal)
                    total_profit = cash_from_sale - initial_investment
                    current_cash = cash_from_sale
                    stocks_owned = 0
            networth[i] = current_cash + stocks_owned * data["Close"].iloc[i]

    if DEBUG:
        print(f'{"DateTime":<25}{"Price":<10}{"Difference":<15}{"%Difference":<15}{"Direction":<10}{"Action":<10}{"Networth":<10}')
        for i in range(len(data)):
            datetime_str = data.index[i].strftime('%Y-%m-%d %H:%M:%S')
            print(f'{datetime_str:<25}{price[i]:<10.2f}{difference[i]:<15.2f}{"{:.2f}%".format(pdifference[i]*100):<15}{direction[i]:<10}{action[i]:<10}{networth[i]:<10.2f}')

    results = ((current_cash + stocks_owned * data["Close"].iloc[-1]) / initial_investment) - 1
    return results


In [101]:
def greedyalgo(data, initial_investment, fee_percent, DEBUG):
    current_cash = initial_investment
    stocks_owned = 0
    fee_decimal = fee_percent / 100.0

    price = [0] * len(data)
    difference = [0] * len(data)
    pdifference = [0] * len(data)
    direction = ["-"] * len(data)
    action = ["HOLD"] * len(data)
    networth = [0] * len(data)

    for i in range(len(data)):
        price[i] = data["Close"].iloc[i]
        if i > 1:
            difference[i] = data["Close"].iloc[i] - data["Close"].iloc[i-1]
            pdifference[i] = (data["Close"].iloc[i] - data["Close"].iloc[i-1]) / data["Close"].iloc[i]
            direction[i] = "UP" if (data["Close"].iloc[i] - data["Close"].iloc[i-1]) > 0 else "DOWN"

            if data["Close"].iloc[i] > data["Close"].iloc[i - 1] and data["Close"].iloc[i-1] < data["Close"].iloc[i - 2]:
                if current_cash > 0:
                    action[i] = "BUY"
                    stocks_to_buy = current_cash / (data["Close"].iloc[i] * (1 + fee_decimal))
                    stocks_owned += stocks_to_buy
                    current_cash = 0

            elif data["Close"].iloc[i] < data["Close"].iloc[i - 1] and data["Close"].iloc[i-1] > data["Close"].iloc[i - 2]:
                if stocks_owned > 0:
                    action[i] = "SELL"
                    cash_from_sale = stocks_owned * data["Close"].iloc[i] * (1 - fee_decimal)
                    total_profit = cash_from_sale - initial_investment
                    current_cash = cash_from_sale
                    stocks_owned = 0
            networth[i] = current_cash + stocks_owned * data["Close"].iloc[i]

    if DEBUG:
        print(f'{"DateTime":<25}{"Price":<10}{"Difference":<15}{"%Difference":<15}{"Direction":<10}{"Action":<10}{"Networth":<10}')
        for i in range(len(data)):
            datetime_str = data.index[i].strftime('%Y-%m-%d %H:%M:%S')
            print(f'{datetime_str:<25}{price[i]:<10.2f}{difference[i]:<15.2f}{"{:.2f}%".format(pdifference[i]*100):<15}{direction[i]:<10}{action[i]:<10}{networth[i]:<10.2f}')

    results = ((current_cash + stocks_owned * data["Close"].iloc[-1]) / initial_investment) - 1
    return results


In [102]:
#this code block allows you to see how the greedy or perfect algorithm would trade for a specific stock for a specific day

import sys
import os
import yfinance as yf
from datetime import datetime, timedelta

#-----------------------------------------------------
#play around with the stock, interval and fee percent
ticker_symbol = "DOGE-USD"
initial_investment = 100
interval = "1m"
fee_percent = 0.1
DEBUG = True
#-----------------------------------------------------


# Get today's date
today = datetime.now()

# Check if today is Saturday or Sunday, and adjust to the most recent Friday
if today.weekday() >= 5:  # 5: Saturday, 6: Sunday
    days_to_subtract = today.weekday() - 4  # Adjust to Friday
    recent_trading_day = today - timedelta(days=days_to_subtract)
else:
    recent_trading_day = today

year = recent_trading_day.year
month = recent_trading_day.month
day = recent_trading_day.day

# Suppress yfinance download output
original_stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')
data = yf.download(ticker_symbol, start=f"{year}-{month:02d}-{day:02d}", end=f"{year}-{month:02d}-{day+1:02d}", interval=interval)
sys.stderr = original_stderr

#-----------------------------------------------------
print("perfect algorithm profit for " + ticker_symbol + " on " + str(year) + "-" + str(month) + "-" + str(day))
perfect_algo_profit = perfectalgo(data, initial_investment, fee_percent, DEBUG)
print("greedy algorithm profit for " + ticker_symbol + " on " + str(year) + "-" + str(month) + "-" + str(day))
greedy_algo_profit = greedyalgo(data, initial_investment, fee_percent, DEBUG)
#-----------------------------------------------------



perfect algorithm profit for DOGE-USD on 2024-8-2
DateTime                 Price     Difference     %Difference    Direction Action    Networth  
2024-08-02 00:00:00      0.12      0.00           0.00%          -         HOLD      0.00      
2024-08-02 00:01:00      0.12      -0.00          -0.20%         DOWN      BUY       99.90     
2024-08-02 00:02:00      0.12      0.00           0.01%          UP        HOLD      99.91     
2024-08-02 00:03:00      0.12      0.00           0.00%          UP        HOLD      99.91     
2024-08-02 00:04:00      0.12      0.00           0.01%          UP        HOLD      99.93     
2024-08-02 00:05:00      0.12      0.00           0.05%          UP        HOLD      99.98     
2024-08-02 00:07:00      0.12      0.00           0.18%          UP        SELL      100.06    
2024-08-02 00:08:00      0.12      -0.00          -0.02%         DOWN      HOLD      100.06    
2024-08-02 00:09:00      0.12      -0.00          -0.01%         DOWN      HOLD      1

In [103]:
# print(data)
print(data.columns)

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


In [104]:
from datetime import datetime
def is_weekend(year, month, day):
    date_obj = datetime(year, month, day)
    return date_obj.weekday() >= 5



In [105]:
import sys
import os
import yfinance as yf

def analyze_day(year, month, day, ticker_symbol, initial_investment, interval, fee_percent, DEBUG):
  try:
    original_stderr = sys.stderr
    sys.stderr = open(os.devnull, 'w')
    data = yf.download(ticker_symbol, start=f"{year}-{month:02d}-{day:02d}", end=f"{year}-{month:02d}-{day+1:02d}", interval=interval)
    sys.stderr = original_stderr

    greedy_algo_profit = greedyalgo(data, initial_investment,fee_percent, DEBUG)
    perfect_algo_profit = perfectalgo(data, initial_investment,fee_percent, DEBUG)

    return greedy_algo_profit, perfect_algo_profit
  except Exception as e:
    print(f"Error processing {ticker_symbol} on {year}-{month}-{day}: {e}")
    return 0, 0


In [106]:
def format_profit_list(profit_list):
    return ' '.join(f'{profit:8.2f}' for profit in profit_list)


In [107]:

import calendar
import logging
from datetime import datetime, timedelta
import pandas as pd
import time

logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger()

current_date = datetime.now()

start_date = current_date - timedelta(days=28)

#-----------------------------------------------------
#play around with the stock, interval and fee percent
ticker_symbol_list = ["AAPL", "GOOG", "MSFT", "IBM", "NVDA", "BTC-USD", "ETH-USD", "DOGE-USD"]
initial_investment = 100
intervals = ["1m", "5m"]
fee_percents = [0, 0.1]
DEBUG = False
#-----------------------------------------------------


for interval in intervals:
    for fee_percent in fee_percents:
        results = []
        print(f"\n{'-'*60}")
        print(f"Summary of Average Daily Returns\nInterval: {interval} | Fee: {fee_percent}%\n{'-'*60}")

        for ticker_symbol in ticker_symbol_list:
            greedyprofit = []
            perfectprofit = []

            start_time = time.time()  # Start timing

            for n in range(28):
                day_date = start_date + timedelta(days=n)
                year = day_date.year
                month = day_date.month
                day = day_date.day

                if day < calendar.monthrange(year, month)[1] and not is_weekend(year, month, day):
                    cgreedyprofit, cperfectprofit = analyze_day(year, month, day, ticker_symbol, initial_investment, interval, fee_percent, DEBUG)
                    greedyprofit.append(cgreedyprofit)
                    perfectprofit.append(cperfectprofit)

            if greedyprofit and perfectprofit:
                average_daily_greedy_profit = sum(greedyprofit) / len(greedyprofit)
                average_daily_perfect_profit = sum(perfectprofit) / len(perfectprofit)

                end_time = time.time()  # End timing
                computation_time = end_time - start_time  # Calculate the computation time

                results.append({
                    "Ticker": ticker_symbol,
                    "Perfect Profit Avg Daily Returns": average_daily_perfect_profit,
                    "Greedy Profit Avg Daily Returns": average_daily_greedy_profit,
                    "Computation Time (s)": computation_time
                })

                # Format the print output
                print(f"Ticker: {ticker_symbol:<10} | Perfect Profit Avg Daily Returns: {average_daily_perfect_profit * 100:>6.2f}% | "
                      f"Greedy Profit Avg Daily Returns: {average_daily_greedy_profit * 100:>6.2f}% | "
                      f"Computation Time: {computation_time:>5.2f} s")

        results_df = pd.DataFrame(results)
        results_df['Perfect Profit Avg Daily Returns'] = results_df['Perfect Profit Avg Daily Returns'].apply(lambda x: f"{x * 100:.2f}%")
        results_df['Greedy Profit Avg Daily Returns'] = results_df['Greedy Profit Avg Daily Returns'].apply(lambda x: f"{x * 100:.2f}%")

        # # Print the results in a formatted table
        # print("\nDetailed Results:\n")
        # print(results_df.to_string(index=False))


------------------------------------------------------------
Summary of Average Daily Returns
Interval: 1m | Fee: 0%
------------------------------------------------------------
Ticker: AAPL       | Perfect Profit Avg Daily Returns:  10.75% | Greedy Profit Avg Daily Returns:  -0.69% | Computation Time:  4.37 s
Ticker: GOOG       | Perfect Profit Avg Daily Returns:  10.42% | Greedy Profit Avg Daily Returns:  -0.61% | Computation Time:  3.47 s
Ticker: MSFT       | Perfect Profit Avg Daily Returns:   9.37% | Greedy Profit Avg Daily Returns:  -0.17% | Computation Time:  3.31 s
Ticker: IBM        | Perfect Profit Avg Daily Returns:   9.09% | Greedy Profit Avg Daily Returns:  -0.42% | Computation Time:  3.44 s
Ticker: NVDA       | Perfect Profit Avg Daily Returns:  25.16% | Greedy Profit Avg Daily Returns:  -0.18% | Computation Time:  4.19 s
Ticker: BTC-USD    | Perfect Profit Avg Daily Returns:  34.35% | Greedy Profit Avg Daily Returns:   4.61% | Computation Time:  8.01 s
Ticker: ETH-USD  